# Data Collection - Harvard
In this notebook, I'll collect data from Harvard Museum.

In [1]:
import pandas as pd
import numpy as np
import requests
import json

import pickle

import time
from PIL import Image

In [2]:
# getting keys
def get_keys(path):
    with open(path) as f: 
        return json.load(f)

In [3]:
path = r"C:\Users\Eunjoo\Documents\.secret"

In [4]:
api_key = get_keys(f"{path}/harvard_mus_api.json")["api_key"]

### Check classifications

In [5]:
url = "https://api.harvardartmuseums.org/classification"

url_params = {
    "apikey": api_key,
}

resp = requests.get(url, params = url_params)
print(resp.status_code)

200


In [20]:
classifications = []

n = int(resp.json()['info']['pages']) # getting the page number 
        
for i in range(n):
    url_params["page"] = i
    print(f"page {i}")
    
    resp = requests.get(url, params = url_params)

    try: 
        classifications.extend(resp.json()['records']) # add it to the list
    except:
        print(f"Error on page {i+1}") # let me know if there's an error

page 0
page 1
page 2
page 3
page 4
page 5


In [22]:
classifications = pd.DataFrame(classifications)

In [29]:
classifications.sort_values(by = 'objectcount', ascending = False)[0:10]

,objectcount,name,id,lastupdate,classificationid
59,84315,Photographs,17,2020-09-15T04:29:44-0400,17
20,27702,Drawings,21,2020-09-15T04:29:44-0400,21
15,6978,Paintings,26,2020-09-15T04:29:44-0400,26
5,6978,Paintings,26,2020-09-15T04:29:44-0400,26
21,6244,Sculpture,30,2020-09-15T04:29:44-0400,30
35,6190,Vessels,57,2020-09-15T04:29:44-0400,57
49,5898,Seals,189,2020-09-15T04:29:44-0400,189
43,4880,Straus Materials,959,2020-09-15T04:29:44-0400,959
25,4800,Fragments,94,2020-09-15T04:29:44-0400,94
48,4388,Manuscripts,185,2020-09-15T04:29:44-0400,185


## Collecting Full Data
It seems like paintings alone could give us enough data for now. I'll collect all paintings data and download each images.

In [166]:
url = "https://api.harvardartmuseums.org/object"

url_params = {
    "apikey": api_key, 
    "classification": 'Paintings'
}

resp = requests.get(url, params = url_params)

resp.status_code

200

In [ ]:
# getting the number of pages
n = int(resp.json()['info']['pages'])
full_db = []

for i in range(219,n):
    url_params['page'] = i
    resp = requests.get(url, params = url_params)
    try: 
        full_db.extend(resp.json()['records'])
    except:
        print(f"error on page {i+1}")
              
    pause = np.random.randint(1, 60)
    print(f'{i+1}/{n} complete. pausing for {pause} secs ...')
    time.sleep(pause) # buffer

In [175]:
#first_set = full_df.copy()

In [197]:
df = pd.DataFrame(full_db)

In [182]:
full_df = pd.concat([first_set, df], axis = 0)

In [198]:
# saving
full_df = df.drop_duplicates(subset = 'id')
filename = 'PKL/raw_data_Harvard_3.pkl'
full_df.to_pickle(filename)

In [6]:
full_df = pd.read_pickle('PKL/raw_data_Harvard_3.pkl')

In [7]:
image_id_set = full_df[['id', 'primaryimageurl']]

In [28]:
# downloading images
def download_img(df): 
    url = str(df['primaryimageurl'])
    print(f'connecting {url}...')
    if url.startswith('http'):
        resp = requests.get(url)
        print(resp.status_code)

        fpath = f"IMAGES/HARVARD/{df['id']}.jpg"
        with open(fpath, 'wb') as fp:
            fp.write(resp.content)

        img = Image.open(fpath)
        img.thumbnail((500, 500))
        img.save(fpath)
    else:
        print('error')
    
    pause = np.random.randint(1, 30)
    print(f'pausing for {pause} secs ...')
    time.sleep(pause) # buffer
    
    return df

In [ ]:
image_id_set.iloc[608:].apply(lambda x: download_img(x), axis = 1)

connecting https://nrs.harvard.edu/urn-3:HUAM:INV012442_dynmc...
200
pausing for 2 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:INV100637_dynmc...
200
pausing for 10 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:77690_dynmc...
200
pausing for 16 secs ...
connecting None...
error
pausing for 12 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:77307_dynmc...
200
pausing for 4 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:INV184194_dynmc...
200
pausing for 21 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:VRS37461_dynmc...
200
pausing for 23 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:769784...
200
pausing for 1 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:VRS37499_dynmc...
200
pausing for 9 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:VRS37402_dynmc...
200
pausing for 10 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:INV101243_dynmc...
200
pausing for 1 secs ...
connecting https://nrs.harvard.edu/urn-3:HUAM:

In [22]:
image_id_set.iloc[606:]

,id,primaryimageurl
782,230513,None
783,230522,https://nrs.harvard.edu/urn-3:HUAM:VRS35602_dynmc
784,230542,https://nrs.harvard.edu/urn-3:HUAM:INV012442_d...
785,230544,https://nrs.harvard.edu/urn-3:HUAM:INV100637_d...
786,230548,https://nrs.harvard.edu/urn-3:HUAM:77690_dynmc
...,...,...
3676,54847,https://nrs.harvard.edu/urn-3:HUAM:INV203062_d...
3677,55219,https://nrs.harvard.edu/urn-3:HUAM:CARP10400_d...
3678,58252,https://nrs.harvard.edu/urn-3:HUAM:755700
3679,59177,https://nrs.harvard.edu/urn-3:HUAM:766363
